# After the Burn: Mapping Vegetation Regrowth After the Chimney Tops 2 Fire in Great Smoky Mountains National Park
Author: Heidi Yoon

In [ ]:
# Import packages
import io
import os
from glob import glob
import sys
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import folium
import h5py
import rioxarray as rxr
from rasterio.plot import plotting_extent
import xarray as xr
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.mask as em

# Check module path
project_path = os.path.abspath(os.path.join('..'))
if project_path not in sys.path:
    sys.path.append(project_path)
    
import modules.reflectance as refl
import modules.landsat as landsat

In [ ]:
# Get NEON reflectance data from NEON API
# Endpoints are data product, site, date, and release year
neon_base = "https://data.neonscience.org/api/v0/data"
data_product = "/DP3.30006.001"
site = "/GRSM"
post_fire_release = "/2017-10?package=basic&release=RELEASE-2022"
post_fire_url = neon_base + data_product + site + post_fire_release
post_fire_data = requests.get(post_fire_url)

# Similarly for the pre-fire data
pre_fire_release = "/2016-06?package=basic&release=RELEASE-2022"
pre_fire_url = neon_base + data_product + site + pre_fire_release
pre_fire_data = requests.get(pre_fire_url)

In [ ]:
# Using API response, find the url corresponding to tile of interest
post_fire_df = pd.json_normalize(post_fire_data.json()['data']['files'])
pre_fire_df = pd.json_normalize(pre_fire_data.json()['data']['files'])

In [ ]:
# Get NEON reflectance data
post_fire_url = post_fire_df[post_fire_df['name']=='NEON_D07_GRSM_DP3_274000_3947000_reflectance.h5']['url'].iloc[0]
pre_fire_url = pre_fire_df[pre_fire_df['name']=='NEON_D07_GRSM_DP3_274000_3947000_reflectance.h5']['url'].iloc[0]

# Define data directory and data paths
data_dir = os.path.join(et.io.HOME, 'earth-analytics', 'ea-2022-capstone-project', 'data')
post_data_path = os.path.join(
    data_dir, 'NEON_GRSM_274000_3947000_201710_reflectance.h5')
pre_data_path = os.path.join(
    data_dir, 'NEON_GRSM_274000_3947000_201606_reflectance.h5')

post_data = refl.download_file(post_data_path, post_fire_url)
pre_data = refl.download_file(pre_data_path, pre_fire_url)

In [ ]:
# Get MTBS fire boundary
fire_bound_url = ('https://github.com/AreteY/post-wildfire-vegetation-change/files/8567790/chimtops2_burn_bndy.zip')
fire_bound_data = et.data.get_data(url=fire_bound_url)

# Set working directory
os.chdir(os.path.join(
    et.io.HOME, 'earth-analytics', 'data'))

chimney_fire_path = os.path.join('earthpy-downloads', 'chimtops2_burn_bndy', 'chimtops2_burn_bndy.shp')

# Open fire and grsm boundary and reproject crs
chimney_fire_bound = gpd.read_file(chimney_fire_path)
chimney_fire_reproj = chimney_fire_bound.to_crs(epsg=32617)

#### Figure 1: Map of the Great Smoky Mountains National Park. 

In [ ]:
# Create dynamic map of the Great Smoky Mountains National Park
# With areas of interest pertaining to the fire
# Define dataframe with aoi coordinates
fire_aoi_df = pd.DataFrame(columns=["name", "latitude", "longitude"],
                           data=[["Chimney Tops Mtn", 35.630653, -83.478226],
                                 ["Gatlinburg, TN", 35.714167, -83.510278],
                                 ["Pigeon Forge, TN", 35.793889, -83.564167]])

fire_map = folium.Map([35.63, -83.48],
                      zoom_start=10)

style = {'fillColor': '#982E40', 'color': '#982E40'} 
folium.GeoJson(data=chimney_fire_bound['geometry'],
               style_function = lambda x: style).add_to(fire_map)

# Add markers for Chimney Tops Fire 2 areas of interest
for name, xy in fire_aoi_df.groupby(["name"]):
    folium.Marker(
        location=[xy.latitude, xy.longitude],
        popup=name,
        icon=folium.Icon()
    ).add_to(fire_map)

fire_map

#### Figure 1 Caption:
From bottom to top, the markers denote: 
* Chimney Tops Mountain
* Gatlinburg, Tennessee 
* Pigeon Forge, Tennessee.

The Chimney Tops 2 Fire began at Chimney Tops Mountain on November 23, 2016 and later headed northwest towards Gatlinburg and Pigeon Forge, Tennessee.

## 2016 Chimney Tops 2 Fire
The [Chimney Tops 2 Fire](https://www.nps.gov/grsm/learn/chimney-tops-2-fire.htm) began in late November in 2016 on Chimney Tops Mountain in Great Smoky Mountains National Park, as shown in Figure 1. Initially, the Chimney Tops 2 Fire was contained within a 400-acre zone for several days. However, due to unseasonably dry and windy conditions, the fire escaped and grew several orders of magnitude in one day! Figure 2 shows the [progression of the fire](https://www.wildfirelessons.net/HigherLogic/System/DownloadDocumentFile.ashx?DocumentFileKey=14eb7c82-02ef-9410-301c-33e967bf8f31&forceDialog=0) and how it grew from 75 to 15,000 acres on November 28. After this day, the Chimney Tops 2 Fire joined other fires in Sevier County (see Fig. 1) and burned until December 22, 2016. [At least 14 people lost their lives, and over 1600 buildings were damaged or destroyed](https://www.cnn.com/2016/12/05/us/tennessee-gatlinburg-wildfires/index.html). Wildfires, such as the Chimney Tops 2 Fire, have become more frequent and more intense. To protect our forests and human communities, we need to understand the effects of fire on our forests.

#### Figure 2: Chimney Tops 2 Fire Progression
![Figure 2](https://raw.githubusercontent.com/AreteY/post-wildfire-recovery/main/graphics/fire_progression.png)
* [Fire progression](https://www.wildfirelessons.net/HigherLogic/System/DownloadDocumentFile.ashx?DocumentFileKey=14eb7c82-02ef-9410-301c-33e967bf8f31&forceDialog=0) by acres burned (upper plot) and weather conditions (lower plot) with inset corresponding to the total burned area.

## Determining the severity of a fire can help managers make better decisions
Determining the severity of a fire can help us understand how the forest will recover and if the fire is likely to happen again. [Fires that burn along the ground are classified as low severity fires, whereas fires that climb up into the canopy and kill most of the trees are classified as high severity fires](https://eos.org/features/a-lidars-eye-view-of-how-forests-are-faring). Recent or regular fires can [change the structure of a forest](http://dx.doi.org/10.1016/j.foreco.2014.10.038) to make the forest more resilient to future fires. Characterizing the burn severity will help forest and fire managers to make better decisions regarding fire.

## Studying fires remotely will allow us to study more fires with greater coverage and efficiency
Scientists have studied the impact of fires by observing the forest floor directly. For example, [Meng et al. 2017](https://www.sciencedirect.com/science/article/pii/S0034425717300159?via%3Dihub) identified and characterized trees, vegetation, and their health in small plots within the burn perimeter of the Crescent Bow Fire. Fieldwork of this kind is time and labor intensive, especially when fires can burn large areas. [Within the national park, the Chimney Tops 2 Fire burned about 11,000 acres](https://www.nps.gov/grsm/learn/chimney-tops-2-fire.htm)! It would be an enormous undertaking for a team to study the entire burned area!

As fires have grown in size and severity, there is a greater need to study more fires. One such way is to use remote sensing technologies to study fires from the air or from space. Remote sensing technologies can provide more land area coverage and more frequent measurements to study fires efficiently.

## Our Study
We compare reflectance data measured by the Operational Land Imager on the [Landsat 8](https://www.usgs.gov/landsat-missions/landsat-8) satellite with those from the [NEON Imaging Spectrometer](https://data.neonscience.org/data-products/DP3.30006.001) on an airplane. Although the NEON spectrometer has high spatial resolution (1-meter), the reflectance data is typically only available once a year. Although the Landsat 8 imager has lower spatial resolution (30-meter), Landsat 8 orbits the earth every 16 days, so there is more data available.

Here is a map of the Chimney Tops 2 Fire Boundary (Figure 3). For our analysis, we will look at one 1-km<sup>2</sup> tile within the burned area using data collected from the air by the NEON Spectrometer and with data collected from space by the Landsat 8 Imager.

In [ ]:
# Open NEON data and store reflectance array and metadata
post_fire_refl, post_fire_metadata = refl.aop_h5refl2array(post_data_path)
pre_fire_refl, pre_fire_metadata = refl.aop_h5refl2array(pre_data_path)

In [ ]:
# Stack and subset bands of interest
pre_fire_stack = refl.stack_subset_bands(
    pre_fire_refl, pre_fire_metadata, (58, 84, 117, 400))

In [ ]:
post_fire_stack = refl.stack_subset_bands(
    post_fire_refl, post_fire_metadata, (58, 84, 117, 400))

#### Figure 3: Map of the Chimney Tops 2 Fire Boundary with Sample NEON Data

In [ ]:
# Plot an example reflectance tile within the fire boundary
f, ax = plt.subplots(figsize=(10, 10))

# Define raster image
plt.imshow(post_fire_refl[:, :, 0], 
           extent=post_fire_metadata['extent'], 
           cmap='Greys', 
           clim=(0, 1200))

# Add fire boundary
chimney_fire_reproj.boundary.plot(ax=ax, color="black")

# Remove scientific notation from tile coordinates
ax.ticklabel_format(useOffset=False, style='plain')

# Set figure title and label axes
ax.set(xlabel='UTM easting (m)',
       ylabel='UTM northing (m)')
rotatexlabels = plt.setp(ax.get_xticklabels(), rotation=90)

# Data source caption
ax.text(.95, .01, "Data: NEON Airborne Observation Platform", va='bottom', ha='right', 
        transform=ax.transAxes, color="blue", fontweight='bold')

plt.show()

#### Figure 3 Caption:
Chimney Tops 2 Fire Boundary ([MTBS Project](https://mtbs.gov/direct-download)) within Great Smoky Mountains National Park with Sample Data Observed by the [NEON spectrometer](https://data.neonscience.org/data-products/DP1.30006.001).

## dNBR identifies healthy and burned vegetation
We can calculate the difference normalized burn ratio, or dNBR, to identify healthy vs burned vegetation before and after the fire using Landsat and NEON data.

#### Figure 4: Difference Normalized Burn Ratio of Landsat Reflectance for the Chimney Tops 2 Fire

In [ ]:
all_landsat_bands = sorted(glob(
    os.path.join('landsat-20170412', '*B[2-7].TIF')))
all_post_fire_bands = sorted(glob(
    os.path.join('landsat-20171005', '*B[2-7].TIF')))
all_pre_fire_bands = sorted(glob(
    os.path.join('landsat-20160409', '*B[2-7].TIF')))

# List of processed bands
all_bands = landsat.process_bands(
    all_landsat_bands, chimney_fire_reproj, stack=False)
all_post_bands = landsat.process_bands(
    all_post_fire_bands, chimney_fire_reproj, stack=False)
all_pre_bands = landsat.process_bands(
    all_pre_fire_bands, chimney_fire_reproj, stack=False)
# Stack of processed bands
landsat_xr = landsat.process_bands(
    all_landsat_bands, chimney_fire_reproj, stack=True)
post_fire_xr = landsat.process_bands(
    all_post_fire_bands, chimney_fire_reproj, stack=True)
pre_fire_xr = landsat.process_bands(
    all_pre_fire_bands, chimney_fire_reproj, stack=True)

In [ ]:
# Define cloud mask values
high_cloud_confidence = (
    em.pixel_flags['pixel_qa']['L8']['High Cloud Confidence'])
cloud = em.pixel_flags['pixel_qa']['L8']['Cloud']
cloud_shadow = em.pixel_flags['pixel_qa']['L8']['Cloud Shadow']

cloud_values = high_cloud_confidence + cloud + cloud_shadow

# Pixel QA Path for Clouds
pixel_qa_path = os.path.join(
    'landsat-20170412',
    'LC08_L2SP_019035_20170412_20200904_02_T1_QA_PIXEL.TIF')
post_qa_path = os.path.join(
    'landsat-20171005', 
    'LC08_L2SP_019035_20171005_20200903_02_T1_QA_PIXEL.TIF')
pre_qa_path = os.path.join(
    'landsat-20160409', 
    'LC08_L2SP_019035_20160409_20200907_02_T1_QA_PIXEL.TIF')

In [ ]:
# Calc nbr and mask for clouds
nbr = landsat.masked_norm_diff(
    all_bands, chimney_fire_reproj, pixel_qa_path, cloud_values, calc='nbr')
post_nbr = landsat.masked_norm_diff(
    all_post_bands, chimney_fire_reproj, post_qa_path, cloud_values, calc='nbr')
pre_nbr = landsat.masked_norm_diff(
    all_pre_bands, chimney_fire_reproj, pre_qa_path, cloud_values, calc='nbr')

# Define dnbr
dnbr = pre_nbr - post_nbr

In [ ]:
# Define the Landsat data plot extent
landsat_extent = plotting_extent(
    landsat_xr[0], landsat_xr.rio.transform())

# Plot dNBR
fig, ax = plt.subplots(figsize=(12, 6))

plot = ax.imshow(dnbr,
                  extent=landsat_extent,
                  cmap='PiYG')

# Color bar settings
cbar = plt.colorbar(plot, aspect=40)
cbar.set_label('Diff Normalized Burn Ratio', rotation=90, labelpad=20)

# Remove scientific notation from tile coordinates
ax.ticklabel_format(useOffset=False, style='plain')

# Set figure title and axes labels
ax.set(
    title='Chimney Tops 2 Fire dNBR\n (Apr. 9, 2016-Oct. 5, 2017)',
    xlabel='UTM easting (m)',
    ylabel='UTM northing (m)')
rotatexlabels = plt.setp(ax.get_xticklabels(), rotation=90)
ax.set(xlim=[274000, 275000], ylim=[3947000, 3948000])

# Data source caption
ax.text(.95, .01, "Source: Landsat", va='bottom',
         ha='right', transform=ax.transAxes, color="blue", fontweight="bold")

plt.show()

#### Figure 4 Caption:
Here, we plot the dNBR for the Chimney Tops 2 Fire using Landsat data. Pink areas represent regrowth after the fire, and green areas represent burn areas that are still recovering. The Landsat data with 30-meter spatial resolution suggests that most of the area within the 1-km<sup>2</sup> tile have signs of regrowth and that about 25% of the tile also has significant regrowth.

#### Figure 5: Difference Normalized Burn Ratio of NEON Reflectance for the Chimney Tops 2 Fire

In [ ]:
# Calculate NBR
post_fire_nbr = refl.calc_norm_diff(
    post_fire_stack[:, :, 2], post_fire_stack[:, :, 3])
pre_fire_nbr = refl.calc_norm_diff(
    pre_fire_stack[:, :, 2], pre_fire_stack[:, :, 3])

# Calculate dNBR
dnbr = pre_fire_nbr - post_fire_nbr

# Plot dNBR
fig, ax = plt.subplots(figsize=(12, 6))

plot = ax.imshow(dnbr,
                  extent=post_fire_metadata['extent'],
                  cmap='PiYG')

# Color bar settings
cbar = plt.colorbar(plot, aspect=40)
cbar.set_label('Diff Normalized Burn Ratio', rotation=90, labelpad=20)

# Remove scientific notation from tile coordinates
ax.ticklabel_format(useOffset=False, style='plain')

# Set figure title and axes labels
ax.set(title='Chimney Tops 2 Fire dNBR\n (June 2016-October 2017)',
       xlabel='UTM easting (m)',
       ylabel='UTM northing (m)')
rotatexlabels = plt.setp(ax.get_xticklabels(), rotation=90)
#ax.set(xlim=[274000, 274200], ylim=[3947300, 3947500])

# Data source caption
ax.text(.95, .01, "Source: NEON Airborne Observation Platform", va='bottom',
         ha='right', transform=ax.transAxes, color="blue", fontweight="bold")

plt.show()

#### Figure 5 Caption: 
Here, we plot the dNBR for the Chimney Tops 2 Fire using NEON data. The NEON data with 1-meter spatial resolution also suggests that most of the area within the 1-km<sup>2</sup> tile have signs of regrowth, but the regrowth does not appear as strong as suggested by the Landsat data. In fact, the 25% of significant regrowth in the Landsat data is characterized as mild regrowth in the NEON data.The higher spatial resolution of the NEON data allows us to measure changes on the order of fire effects on the ground. Availability of higher resolution remote measurements will allow us to study fires more efficiently and with greater accuracy.

## Next steps
We are also working on a spectral analysis with NEON reflectance data and field data. Learn more about our post-wildfire recovery study below! 
* https://github.com/AreteY/post-wildfire-recovery  